In [2]:
import sagemaker
import boto3
from sagemaker import get_execution_role

In [3]:
region = boto3.Session().region_name
sm_rt = boto3.Session().client('runtime.sagemaker', region_name=region)

ep_name = 'eeg-outlier-model'
id_name = ''
target = 'Eye'

file_name = "EEG-outlier-test.csv"
test_file_name = "EEG-outlier-test-no-label.csv"

kaggle_file_name = ""
kaggle_test_file_name = ""
kaggle_pred_file_name = ""

In [4]:
import pandas as pd
import numpy as np

df = pd.read_csv(file_name)
df = df.drop([target],axis=1) 
print(df.head())
print(df.shape)

df.to_csv(test_file_name, index=False)

       AF3       F7       F3      FC5       T7       P7       O1       O2  \
0  4343.59  3971.79  4265.13  4092.31  4326.67  4618.97  4100.00  4626.15   
1  4431.79  4072.31  4304.62  4154.36  4350.77  4621.03  4077.44  4601.54   
2  4316.41  4037.44  4247.18  4134.87  4327.69  4592.82  4048.21  4598.46   
3  4285.13  3996.41  4260.51  4108.72  4333.33  4617.95  4064.10  4625.13   
4  4288.72  4018.46  4267.69  4131.79  4338.97  4629.23  4086.15  4616.41   

        P8       T8      FC6       F4       F8      AF4  
0  4205.64  4227.69  4238.46  4309.74  4654.36  4403.08  
1  4211.28  4275.90  4249.74  4321.54  4685.64  4476.41  
2  4190.26  4216.92  4177.44  4257.95  4588.21  4365.64  
3  4203.08  4229.23  4201.54  4281.03  4606.15  4354.36  
4  4191.79  4221.03  4196.41  4270.77  4590.26  4336.92  
(4494, 14)


In [10]:
f = open(test_file_name)
count = 0
pred_label = []
pred_neg_prpb = []
pred_pos_prpb = []

for line in f:
    if count==0:
        count=1
        continue
    response = sm_rt.invoke_endpoint(EndpointName=ep_name, ContentType='text/csv', Accept='text/csv', Body=line.encode('utf-8'))
    response = response['Body'].read().decode("utf-8")
    #print(response)
    pred_label.append(int(response[0]))
    #ind1 = response.index("[")
    #ind2 = response.index(",", ind1)
    #neg_prob = float(response[ind1+1:ind2])
    #print(neg_prob)
    #pred_neg_prpb.append(neg_prob)
    #ind1 = response.index("]", ind2)
    #pos_prob = float(response[ind2+2:ind1])
    ind1 = response.index(",")
    neg_prob = float(response[ind1+1:])
    pred_neg_prpb.append(neg_prob)
    pos_prob = 1 - neg_prob
    #print(pos_prob)
    pred_pos_prpb.append(pos_prob)
    count = count + 1


In [12]:
print(pred_label[:20])
print(pred_neg_prpb[:20])
print(pred_pos_prpb[:20])


import pandas as pd
import numpy as np
pd = pd.read_csv(file_name)
y_true = np.array(pd[target])
y_pred = np.array(pred_label)
print("test accuracy = {}".format(np.mean(y_true==y_pred)))

from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y_true, pred_neg_prpb, pos_label=1)
print("test auc = {}".format(metrics.auc(fpr, tpr)))

[1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1]
[0.9957200884819031, 0.0099606579169631, 0.8584847450256348, 0.008934120647609234, 0.3356174826622009, 0.0017440435476601124, 0.005327907856553793, 0.11146163195371628, 0.8982624411582947, 0.9419276714324951, 0.9875872731208801, 0.0022254008799791336, 0.009246845729649067, 0.008924937807023525, 0.003984182607382536, 0.04974837601184845, 0.017799168825149536, 0.9748295545578003, 0.026122814044356346, 0.9790793657302856]
[0.004279911518096924, 0.9900393420830369, 0.14151525497436523, 0.9910658793523908, 0.6643825173377991, 0.9982559564523399, 0.9946720921434462, 0.8885383680462837, 0.10173755884170532, 0.05807232856750488, 0.012412726879119873, 0.9977745991200209, 0.9907531542703509, 0.9910750621929765, 0.9960158173926175, 0.9502516239881516, 0.9822008311748505, 0.025170445442199707, 0.9738771859556437, 0.020920634269714355]
test accuracy = 0.9425901201602136
test auc = 0.98744390848387


In [12]:
import pandas as pd
import numpy as np

df = pd.read_csv(kaggle_file_name)

kaggle_test_ids = np.array(df[id_name])

df = df.drop([id_name],axis=1) 
print(df.head())
print(df.shape)

df.to_csv(kaggle_test_file_name, index=False)

   RESOURCE  MGR_ID  ROLE_ROLLUP_1  ROLE_ROLLUP_2  ROLE_DEPTNAME  ROLE_TITLE  \
0     78766   72734         118079         118080         117878      117879   
1     40644    4378         117961         118327         118507      118863   
2     75443    2395         117961         118300         119488      118172   
3     43219   19986         117961         118225         118403      120773   
4     42093   50015         117961         118343         119598      118422   

   ROLE_FAMILY_DESC  ROLE_FAMILY  ROLE_CODE  
0            118177        19721     117880  
1            122008       118398     118865  
2            301534       249618     118175  
3            136187       118960     120774  
4            300136       118424     118425  
(58921, 9)


In [14]:
f = open(kaggle_test_file_name)
count = 0
kaggle_pred = []

for line in f:
    if count == 0:
        count = 1
        continue
    response = sm_rt.invoke_endpoint(EndpointName=ep_name, ContentType='text/csv', Accept='text/csv', Body=line.encode('utf-8'))
    response = response['Body'].read().decode("utf-8")
    test_id = kaggle_test_ids[count-1]
    pred = int(response[0])
    kaggle_pred.append([test_id, pred])
    count = count + 1

kaggle_pred_df = pd.DataFrame(kaggle_pred, columns = [id_name, target])

kaggle_pred_df.to_csv(kaggle_pred_file_name, index=False)

In [21]:
print(kaggle_pred_df.head())
print(kaggle_pred_df.shape)

   id  ACTION
0   1       1
1   2       1
2   3       1
3   4       1
4   5       1
(58921, 2)


In [22]:
import boto3
sm = boto3.client('sagemaker')
#sm.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm.delete_endpoint(EndpointName=ep_name)

{'ResponseMetadata': {'RequestId': '3adbc464-12f4-4c5b-af4a-1bad8f737f35',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '3adbc464-12f4-4c5b-af4a-1bad8f737f35',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Wed, 24 Mar 2021 07:51:25 GMT'},
  'RetryAttempts': 0}}